# Technical Analysis Functions and New DB

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
from finta import TA

import matplotlib as plot
import mplfinance as mpf
import plotly.graph_objects as go


In [ ]:
# Get some stock prices from our Prices Database
# Load a list of ticker symbols to load
con1 = sqlite3.connect("TSX_Prices.sqlite")
#con2 = sqlite3.connect("TSX_Analysis.sqlite")

ticker = "SHOP"
sql = f"SELECT * FROM 'prices_daily' WHERE Ticker = '{ticker}' ORDER BY date ASC"
df = pd.read_sql_query(sql, con1)
df.drop(['index'], axis=1, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
df.set_index("Date")
df.tail(50)

In [ ]:
df.Low[1808]

In [ ]:
# Calculate if support at dtae l using n1 and n2 before and after target date (l)
def support(df, l, n1, n2): 
    for i in range(l-n1+1, l+1):
        if (df.Low[i] > df.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if (df.Low[i] < df.Low[i-1] ):
            return 0
    return 1

def resistance(df, l ,n1, n2):
    for i in range(l-n1+1, l+1):
        if (df.High[i] < df.High[i-1]):
            return 0
    for i in range(l+1, l+n2+1):
        if ( df.High[i] > df.High[i-1] ):
            return 0
    return 1

sr = []
n1 = 3
n2 = 2

# FINISH THIS ?????

In [ ]:
df["previous_close"] = df["Close"].shift(1)
df["returns_manual"] = (df["Close"] / df["previous_close"]) - 1
df["change %"] = df["Close"] - df["previous_close"]
df["returns"] = df["Close"].pct_change()
# LN(1+return) -> EXP(sum of LN) - 1
df["log_return"] = np.log(1 + df["returns"])
df["cum_log"] = np.exp(df["log_return"].cumsum()) - 1
df["SMA200"] = df["Close"].rolling(window=250).mean()
df["Trend"] = np.where(df["Close"] > df["SMA200"], "Up", "Down")
df.tail(10)

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df_t['Date'],
                open=df_t['Open'],
                high=df_t['High'],
                low=df_t['Low'],
                close=df_t['Close'])
                ])

fig.add_trace(go.Scatter(x=df_t['Date'], 
                         y=df_t['SMA200'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=4), 
                         name='SMA 200'))


fig.show()

In [16]:
import pandas as pd
import sqlite3

con = sqlite3.connect("testing.sqlite")
data = {"Open" : ["1.01", "1.02", "1.03"], "Close" : ["2.01", "2.02", "2.03"], "Volume" : ["30001", "30002", "30003"], "Ticker" : ["TSLA", "NSFT", "AMZN"]}
#columns = ["Open", "Close", "Volume"]
dates = ["2021-01-13", "2021-01-12", "2021-01-11"]
df = pd.DataFrame(data, index=dates)
df.index = pd.to_datetime(df.index, infer_datetime_format=True)
df.to_sql("test", con, index="Date", if_exists="replace")
df

,Open,Close,Volume,Ticker
2021-01-13,1.01,2.01,30001,TSLA
2021-01-12,1.02,2.02,30002,NSFT
2021-01-11,1.03,2.03,30003,AMZN


In [17]:

df.drop("Ticker", axis=1, inplace=True)
df.to_sql("test", con, index="Date", if_exists="append")
df

,Open,Close,Volume
2021-01-13,1.01,2.01,30001
2021-01-12,1.02,2.02,30002
2021-01-11,1.03,2.03,30003


In [ ]:
#shop = prices_per_ticker.get_group("SHOP").sort_index().loc["2021"]
shop = prices_per_ticker.get_group("SHOP")

#plot.rcParams['figure.figsize'] = [100, 7]
#shop["close"].plot()

mpf.plot(shop["2021-01":"2021-04"], type="candle",mav=(200), volume=True, figratio=(40,15), style="yahoo")


In [ ]:

#mpf.plot(prices_2["2021-01":"2021-04"], type="candle",mav=(20), volume=True, figratio=(40,15), style="yahoo")
mpf.plot(prices_2["2021":"2022"], type="candle",mav=(20), volume=True, figratio=(40,15), style="yahoo")

In [ ]:
plot_data = prices_2["2019":"2022"]
fig = go.Figure(data=[go.Candlestick(
    x     = plot_data.index,
    open  = plot_data["open"],
    high  = plot_data["high"],
    low   = plot_data["low"],
    close = plot_data["close"]
)])

fig.add_trace(go.Scatter(
    x    = plot_data.index,
    y    = plot_data["SMA100"],
    line = dict(color = "#e0e0e0"),
    name = "SMA100"
))

fig.update_layout(xaxis_rangeslider_visible=True, template="plotly_dark")
fig.update_layout(yaxis_title = "Price CDN$", xaxis_title="Date")
fig.show()